In [1]:
!python3 -m pip install -r requirements.txt 


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\MY PC\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [48]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pke
import traceback
from flashtext import KeywordProcessor
# import sys
# !pip install rake_nltk
from rake_nltk import Rake
# !pip install pke
# !pip install multipartiterank

# sys.path.append('pke-master/pke-master/pke/unsupervised/graph_based')
# from multipartiterank import MultipartiteRank

class prep:
    data=pd.read_csv("OS_dataset - Dataset.csv")
    def search_word(self,sentence, word):
        sentence_lower = sentence.lower()
        word_lower = word.lower()
        if word_lower in sentence_lower:
            return True
        else:
            return False
        
#     lis=data['Topic']
#     w="os"
#     indi=[]
#     i=0
#     for t in lis:
#         if search_word(t,w):
#             indi.append(i)
#         i=i+1
            
    summary_model = T5ForConditionalGeneration.from_pretrained('t5-base')
    summary_tokenizer = T5Tokenizer.from_pretrained('t5-base')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    summary_model = summary_model.to(device)
    
    def postprocesstext(self,content):
        final=""
        for sent in sent_tokenize(content):
            sent = sent.capitalize()
            final = final +" "+sent
        return final
    
    def summarizer(self,text,model,tokenizer):
        text = text.strip().replace("\n"," ")
        text = "summarize: "+text
        # print (text)
        max_len = 512
        encoding = tokenizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

        input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

        outs = model.generate(input_ids=input_ids,
                                        attention_mask=attention_mask,
                                        early_stopping=True,
                                        num_beams=3,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=2,
                                        min_length = 75,
                                        max_length=300)
        dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]
        summary = dec[0]
        summary = self.postprocesstext(summary)
        summary= summary.strip()
        return summary
   
   
    def get_keywords(self,text):
#     text='Natural Language Generation (NLG) is a branch of artificial intelligence (AI) and natural language processing (NLP) that focuses on generating human-like text or speech based on given input or data. NLG algorithms aim to produce coherent, contextually relevant, and linguistically appropriate output that resembles human-generated language.NLG systems utilize various techniques and approaches to generate text. These may include rule-based systems, statistical models, or more advanced deep learning architectures such as recurrent neural networks (RNNs) and transformer models like GPT (Generative Pre-trained Transformer). These models learn from large amounts of text data to capture patterns, semantics, and stylistic nuances in language.'
        r=Rake(punctuations=[')','(',',','.',':',').','),'])
        r.extract_keywords_from_text(text)
    #     print(r.get_ranked_phrases_with_scores())
        implist=[]
        for rating,keyword in r.get_ranked_phrases_with_scores():
            if rating>=4:
                implist.append(keyword)
        return implist 
    def m(self):
        question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
#         question_model = question_model
        return question_model
    def l(self):
        question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
        return question_tokenizer
    
    
    def get_question(self,context,answer,model,tokenizer):
        text = "context: {} answer: {}".format(context,answer)
        encoding = tokenizer.encode_plus(text,max_length=384, pad_to_max_length=False,truncation=True, return_tensors="pt")
        input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
        outs = model.generate(input_ids=input_ids,
                                        attention_mask=attention_mask,
                                        early_stopping=True,
                                        num_beams=5,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=2,
                                        max_length=72)
        dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]
        Question = dec[0].replace("question:","")
        Question= Question.strip()
        return Question


[nltk_data] Downloading package punkt to C:\Users\MY
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to C:\Users\MY
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\MY
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\MY
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement multipartiterank (from versions: none)
ERROR: No matching distribution found for multipartiterank
C:\Users\MY PC\AppData\Roaming\Python\Python39\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [3]:
import pickle

In [49]:
obj=prep()

In [50]:
pickle.dump(obj,open('test.pkl','wb'))

In [51]:
x=pickle.load(open('test.pkl','rb'))

In [52]:
x.search_word("hello h r u","hello")

True

In [53]:
d=pd.read_csv("OS_dataset - Dataset.csv")

In [54]:
x.postprocesstext("hello")

' Hello'

In [55]:
que=x.m()

In [56]:
qt=x.l()

In [58]:
text=d['Text'][0]
word="os"
summarized_text = x.summarizer(text,x.summary_model,x.summary_tokenizer)
print(summarized_text)
imp_keywords = x.get_keywords(text)
for answer in imp_keywords:
    ques = x.get_question(summarized_text,answer,que,qt)
    print(ques)
#     questins.append(ques)
#     answers.append(answer.capitalize())

Our tutorial is designed for beginners, professionals and gate aspirants. The content is described in detailed manner and has the ability to answer most of your queries.the tutorial also contains the numerical examples based on previous year gate questions which will help you to address the problems in a practical manner.the operating system tutorial has been divided into various parts depending on its functions such as process management, process synchronization, deadlocks and file management.
What is the operating system tutorial divided into?
What are the numerical examples based on?
What is the main function of the operating system tutorial?
How is the operating system tutorial divided?
What does the operating system tutorial help you to do?
What kind of examples does the operating system tutorial contain?
What numerical examples does the operating system tutorial contain?
What part of the operating system tutorial has been divided into various parts?
What has been divided into var

In [77]:
import nltk

def get_keywords(text):
#     text='Natural Language Generation (NLG) is a branch of artificial intelligence (AI) and natural language processing (NLP) that focuses on generating human-like text or speech based on given input or data. NLG algorithms aim to produce coherent, contextually relevant, and linguistically appropriate output that resembles human-generated language.NLG systems utilize various techniques and approaches to generate text. These may include rule-based systems, statistical models, or more advanced deep learning architectures such as recurrent neural networks (RNNs) and transformer models like GPT (Generative Pre-trained Transformer). These models learn from large amounts of text data to capture patterns, semantics, and stylistic nuances in language.'
    r=Rake(punctuations=[')','(',',','.',':',').','),'])
    r.extract_keywords_from_text(text)
#     print(r.get_ranked_phrases_with_scores())
    implist=[]
    for rating,keyword in r.get_ranked_phrases_with_scores():
        if rating>=4:
            implist.append(keyword)
    return implist        
   

Defaulting to user installation because normal site-packages is not writeable
  Using cached rake_nltk-1.0.6-py3-none-any.whl (9.1 kB)
[(32.75, 'may include rule - based systems'), (24.75, 'generative pre - trained transformer'), (23.5, 'nlg systems utilize various techniques'), (23.25, 'resembles human - generated language'), (22.75, 'generating human - like text'), (16.0, 'advanced deep learning architectures'), (15.666666666666666, 'transformer models like gpt'), (9.0, 'recurrent neural networks'), (9.0, 'nlg algorithms aim'), (9.0, 'natural language processing'), (9.0, 'natural language generation'), (9.0, 'linguistically appropriate output'), (6.0, 'speech based'), (5.0, 'generate text'), (4.666666666666666, 'statistical models'), (4.666666666666666, 'models learn'), (4.5, 'text data'), (4.0, 'stylistic nuances'), (4.0, 'produce coherent'), (4.0, 'large amounts'), (4.0, 'given input'), (4.0, 'contextually relevant'), (4.0, 'capture patterns'), (4.0, 'artificial intelligence'), (3.

In [ ]:
 def get_nouns_multipartite(self,content):
        out=[]
        try:
            extractor = MultipartiteRank()
            extractor.load_document(input=content,language='en')
            pos = {'PROPN','NOUN'}
            stoplist = list(string.punctuation)
            stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
            stoplist += stopwords.words('english')
            extractor.candidate_selection(pos=pos)
            extractor.candidate_weighting(alpha=1.1,
                                        threshold=0.75,
                                        method='average')
            keyphrases = extractor.get_n_best(n=15)
            for val in keyphrases:
                out.append(val[0])
        except:
            out = []
            traceback.print_exc()
        return out
    
    def get_keywords(self,originaltext,summarytext):
        keywords = self.get_nouns_multipartite(originaltext)
        print ("keywords unsummarized: ",keywords)
        keyword_processor = KeywordProcessor()
        for keyword in keywords:
            keyword_processor.add_keyword(keyword)
        keywords_found = keyword_processor.extract_keywords(summarytext)
        keywords_found = list(set(keywords_found))
        print ("keywords_found in summarized: ",keywords_found)
        important_keywords =[]
        for keyword in keywords:
            if keyword in keywords_found:
                important_keywords.append(keyword)
        return important_keywords[:4]